In [1]:
import sys
sys.path.append('..')

### Set API key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
OPENAI_API_KEY = config['OPENAI']['OPENAI_API_KEY']
YOUTUBE_KEY = config['YOUTUBE']['YOUTUBE_API_KEY']
NAVER_CLIENT_ID = config['NAVER']['NAVER_CLIENT_ID']
NAVER_CLIENT_SECRET = config['NAVER']['NAVER_CLIENT_SECRET']
GOOGLE_SEARCH_KEY = config['GOOGLE']['GOOGLE_API_KEY']
CSE_ID = config['GOOGLE']['CSE_ID']

os.environ.update({'OPENAI_API_KEY': OPENAI_API_KEY})
os.environ.update({'YOUTUBE_KEY': YOUTUBE_KEY})
os.environ.update({'NAVER_CLIENT_ID': NAVER_CLIENT_ID})
os.environ.update({'NAVER_CLIENT_SECRET': NAVER_CLIENT_SECRET})
os.environ.update({'GOOGLE_SEARCH_KEY': GOOGLE_SEARCH_KEY})
os.environ.update({'CSE_ID': CSE_ID})

In [3]:
import time
import asyncio
from typing import List

from tools.search_tool import SearchTool
from models.llm.chain import KeywordsChain, DraftChain, TableChain
from models.draft_generator import DraftGeneratorInstance
from models.keywords_generator import KeywordsGeneratorInstance
from models.table_generator import TableGeneratorInstance
from modules import Project

In [4]:
verbose = False

table_chain_prompt_template_path = '../../openai_skt/models/templates/table_prompt_template.txt'
keywords_chain_prompt_template_path = '../../openai_skt/models/templates/keywords_prompt_template.txt'
draft_chain_prompt_template_path = '../../openai_skt/models/templates/draft_prompt_template.txt'

with open(table_chain_prompt_template_path, 'r') as f:
    table_chain_prompt_template = f.read()
with open(keywords_chain_prompt_template_path, 'r') as f:
    keywords_chain_prompt_template = f.read()
with open(draft_chain_prompt_template_path, 'r') as f:
    draft_chain_prompt_template = f.read()
table_chain = TableChain(table_template=table_chain_prompt_template, input_variables=["purpose"], verbose=verbose)
keywords_chain = KeywordsChain(keywords_template=keywords_chain_prompt_template, input_variables=["purpose", "table"], verbose=verbose)
draft_chain = DraftChain(draft_template=draft_chain_prompt_template, input_variables=["purpose", "draft", "database", "single_table", "table"], verbose=verbose)

table_generator_instance = TableGeneratorInstance(table_chain=table_chain)
keywords_generator_instance = KeywordsGeneratorInstance(keywords_chain=keywords_chain)
draft_generator_instance = DraftGeneratorInstance(draft_chain=draft_chain)

search_tool = SearchTool()

In [5]:
project = Project(
    project_id="test_2", 
    table_generator_instance=table_generator_instance, 
    keywords_generator_instance=keywords_generator_instance, 
    draft_generator_instance=draft_generator_instance, 
    search_tool=search_tool
)

In [6]:
purpose = project.set_purpose(purpose="디지털 자산과 비트코인")
print('purpose: ', purpose)

purpose:  디지털 자산과 비트코인


In [7]:
table = project.get_table()
print('table: \n', table)

table: 
 1. 서론
- 디지털 자산의 개념과 중요성
- 비트코인의 등장과 역할
2. 비트코인의 작동 원리
- 블록체인 기술과 분산원장
- 비트코인의 채굴과 거래 과정
3. 비트코인의 장점과 단점
- 익명성과 보안성
- 가치의 변동성과 부정적인 영향
4. 비트코인의 활용과 영향
- 전자상거래와 결제 시스템에서의 활용
- 글로벌 경제와 금융 시스템에 미치는 영향
5. 비트코인과 법적인 쟁점
- 국가별 비트코인 규제와 법적인 문제
- 금융 기관과의 관계와 협력
6. 비트코인의 미래 전망
- 디지털 자산 시장의 발전과 비트코인의 위치
- 글쓰기 주제와 관련된 개인적인 견해나 생각
7. 결론
- 디지털 자산과 비트코인의 중요성과 잠재력


In [8]:
keywords = project.get_keywords()
print('keywords: ', keywords)

keywords:  ['디지털 자산 개념', '비트코인 역할', '블록체인 기술', '비트코인 채굴 과정', '비트코인 거래 과정', '비트코인 장점', '비트코인 단점', '비트코인 전자상거래 활용', '비트코인 결제 시스템 활용', '비트코인 글로벌 경제 영향', '비트코인 법적인 쟁점', '비트코인 국가별 규제', '비트코인 금융 기관 관계', '비트코인 미래 전망', '디지털 자산 시장 발전', '비트코인의 위치', '디지털 자산 중요성', '비트코인 잠재력']


In [9]:
files = project.search_keywords()

통계청 API 검색 요청에 실패하였습니다., status_code: 502


In [13]:
api_name = 'youtube'
print(len(files[api_name]))

90


In [16]:
file_num = 0
for file in files.values():
    file_num += len(file)
print(file_num, '개의 파일이 검색되었습니다.')

280 개의 파일이 검색되었습니다.


In [ ]:
database = project.parse_files_to_embedchain() # 280files take 14min 45sec
print('database: ', database)

In [26]:
project.draft_generator_instance = draft_generator_instance

In [27]:
draft = project.get_draft()
print('draft: ', draft) # 5min 47sec

draft:  # 서론

디지털 자산은 현대 사회에서 점차적으로 중요성을 갖추고 있는 개념입니다. 디지털 자산은 전자적인 형태로 존재하며, 재산적 가치를 가지고 있으며 사용할 권리가 있는 무형자산입니다. 이러한 디지털 자산은 기존의 재산 개념을 넘어서며, 새로운 경제 시스템과 금융 시장을 형성하고 있습니다.

비트코인은 디지털 자산 중에서도 가장 잘 알려진 암호화폐입니다. 2009년에 등장한 비트코인은 블록체인 기술을 기반으로 작동하며, 분산원장을 통해 거래 기록을 관리합니다. 비트코인은 중앙 기관이나 정부의 감독을 받지 않고 운영되며, 익명성과 보안성을 갖추고 있습니다.

비트코인은 전 세계적으로 거래되고 있으며, 전자상거래와 결제 시스템에서 널리 활용되고 있습니다. 또한 글로벌 경제와 금융 시스템에도 영향을 미치고 있으며, 새로운 경제 모델과 기회를 창출하고 있습니다.

하지만 비트코인은 가치의 변동성과 부정적인 영향을 가지고 있습니다. 이러한 문제점들은 국가별로 다양한 규제와 법적인 쟁점을 야기하고 있습니다. 따라서 비트코인의 발전과 활용에는 법적인 쟁점을 해결하고 금융 기관과의 협력이 필요합니다.

이번 글에서는 디지털 자산의 개념과 중요성에 대해 알아보고, 비트코인의 등장과 역할에 대해 살펴보겠습니다. 이를 통해 디지털 자산과 비트코인에 대한 이해를 높이고, 미래의 발전 가능성을 탐구해보겠습니다.
## 비트코인의 작동 원리

### 블록체인 기술과 분산원장

비트코인은 블록체인 기술을 기반으로 작동합니다. 블록체인은 거래 내역을 분산 저장하며, 이를 위해 네트워크에 참여하는 다수의 노드들이 거래 내역을 확인하고 검증하는 과정을 거칩니다. 이 과정을 통해 거래 내역의 위변조를 방지하고, 신뢰성과 보안성을 확보할 수 있습니다.

블록체인은 블록이라는 단위로 거래 내역을 저장합니다. 각 블록은 이전 블록과 연결되어 있으며, 이를 체인 형태로 구성합니다. 따라서 블록체인은 이전의 거래 내역을 모두 포함하고 있으며, 변경이 발생하면 모든 노드들이 이를 확인하고

In [28]:
project.save_instance()

saved user instance to ./user\test_2\user_instance.json
saved draft to ./user\test_2\draft_0.md
saved database to ./user\test_2\database.json
